In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
cases = pd.read_csv('csvs/justice2.csv')
judges = pd.read_csv('csvs/table_of_justices.csv')
presidents = pd.read_csv('csvs/presidents.csv')

In [3]:
judges['start_term'] = pd.to_datetime(judges['Supreme Court Term Start'])
judges['end_term'] = pd.to_datetime(judges['Supreme Court Term End'].replace("--","01-Mar-24"))

/var/folders/n1/t8qkhtv55nd_gfb_2gwkl_vw0000gn/T/ipykernel_60501/898508096.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  judges['end_term'] = pd.to_datetime(judges['Supreme Court Term End'].replace("--","01-Mar-24"))


In [4]:
judges.dtypes

Index                                int64
Justice Name                        object
Supreme Court Term Start            object
Supreme Court Term End              object
Appointing President                object
Notable Opinion(s)                  object
start_term                  datetime64[ns]
end_term                    datetime64[ns]
dtype: object

In [5]:
clean_judge_df = judges[['Justice Name','start_term', 'end_term', 'Appointing President']]

In [6]:
clean_judge_df['start_year'] = clean_judge_df['start_term'].dt.to_period('Y')
clean_judge_df['end_year'] = clean_judge_df['end_term'].dt.to_period('Y')

/var/folders/n1/t8qkhtv55nd_gfb_2gwkl_vw0000gn/T/ipykernel_60501/426193693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_judge_df['start_year'] = clean_judge_df['start_term'].dt.to_period('Y')
/var/folders/n1/t8qkhtv55nd_gfb_2gwkl_vw0000gn/T/ipykernel_60501/426193693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_judge_df['end_year'] = clean_judge_df['end_term'].dt.to_period('Y')


In [7]:
clean_judge_df

,Justice Name,start_term,end_term,Appointing President,start_year,end_year
0,"Jackson, Ketanji Brown(Associate Justice)",2022-06-30,2024-03-01,"Biden, Joseph R.",2022,2024
1,"Coney Barrett, Amy(Associate Justice)",2020-10-27,2024-03-01,"Trump, Donald J.",2020,2024
2,"Kavanaugh, Brett M.(Associate Justice)",2017-10-06,2024-03-01,"Trump, Donald J.",2017,2024
3,"Gorsuch, Neil M.(Associate Justice)",2017-04-10,2024-03-01,"Trump, Donald J.",2017,2024
4,"Kagan, Elena(Associate Justice)",2010-08-07,2024-03-01,"Obama, Barack H.",2010,2024
...,...,...,...,...,...,...
116,"Rutledge, John(Associate Justice)",1790-02-15,1791-03-05,"Washington, George",1790,1791
117,"Cushing, William(Associate Justice)",1790-02-02,1810-09-13,"Washington, George",1790,1810
118,"Blair, John(Associate Justice)",1790-02-02,1795-10-25,"Washington, George",1790,1795
119,"Jay, John(Chief Justice)",1789-10-19,1795-06-29,"Washington, George",1789,1795


In [8]:
cases['term'][48]


1972

In [9]:
cases.term = pd.to_datetime(cases['term']).dt.to_period('Y')

In [10]:
clean_judge_df['president_last_name'] = clean_judge_df['Appointing President'].str.extract('(\w*),', expand=True) 

/var/folders/n1/t8qkhtv55nd_gfb_2gwkl_vw0000gn/T/ipykernel_60501/3928696604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_judge_df['president_last_name'] = clean_judge_df['Appointing President'].str.extract('(\w*),', expand=True)


In [11]:
clean_judge_df.dtypes

Justice Name                    object
start_term              datetime64[ns]
end_term                datetime64[ns]
Appointing President            object
start_year               period[A-DEC]
end_year                 period[A-DEC]
president_last_name             object
dtype: object

In [12]:
presidents['president_last_name']= presidents['President '].str.extract(r'\s(\w*)$') 

In [13]:
presidents[['president_last_name','Party ']]

,president_last_name,Party
0,Washington,Independent
1,Adams,Federalist
2,Jefferson,Democratic-Republican
3,Madison,Democratic-Republican
4,Monroe,Democratic-Republican
5,Adams,Democratic-Republican/National Republican
6,Jackson,Democratic
7,Buren,Democratic
8,Harrison,Whig
9,Tyler,Whig


In [14]:
judge_parties = pd.merge(presidents,clean_judge_df)
judge_parties[['start_year', 'end_year', 'Party ']]

,start_year,end_year,Party
0,1796,1800,Independent
1,1796,1811,Independent
2,1795,1795,Independent
3,1793,1806,Independent
4,1791,1793,Independent
...,...,...,...
142,2009,2024,Democratic
143,2020,2024,Republican
144,2017,2024,Republican
145,2017,2024,Republican


In [30]:
a = cases.term.astype(str).astype(int).values
bh = judge_parties.start_year.astype(str).astype(int).values
bl = judge_parties.end_year.astype(str).astype(int).values

i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))

pd.concat([
    cases.loc[i, :].reset_index(drop=True),
    judge_parties.loc[j, :].reset_index(drop=True)
    ], axis=1)

,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,...,issue_area,President,Party,president_last_name,Justice Name,start_term,end_term,Appointing President,start_year,end_year


In [50]:


a = c2.term.astype(str).astype(int)
bh = j2.start_year.astype(str).astype(int)
bl = j2.end_year.astype(str).astype(int)

for court_case in cases:
    if a >= bl and a <= bh:
        if j2s['Party '] == "Republican":
            c2.reps += 1

            

AttributeError: 'function' object has no attribute 'term'

In [51]:

c2 = cases.reset_index
j2 = judge_parties.reset_index

c2

<bound method DataFrame.reset_index of       Unnamed: 0     ID                                 name  \
0              0  50606                          Roe v. Wade   
1              1  50613                  Stanley v. Illinois   
2              2  50623              Giglio v. United States   
3              3  50632                         Reed v. Reed   
4              4  50643                 Miller v. California   
...          ...    ...                                  ...   
3298        3298  63324    United States v. Palomar-Santiago   
3299        3299  63323               Terry v. United States   
3300        3300  63331              United States v. Cooley   
3301        3301  63332                   Florida v. Georgia   
3302        3302  63335  PennEast Pipeline Co. v. New Jersey   

                                          href    docket  term  \
0        https://api.oyez.org/cases/1971/70-18     70-18  1970   
1      https://api.oyez.org/cases/1971/70-5014   70-5014  19

In [36]:
combined = pd.merge(cases, judge_parties, left_on=lambda x: cases2['term'].iloc[x] >= judge_parties2['start_year'].iloc[x] 
                                           and cases2['term'].iloc[x] <= judge_parties2['end_year'].iloc[x],
                                           right_index=True, how='left')

KeyError: <function <lambda> at 0x11a760540>

In [37]:
pd.concat([
    cases.loc[i, :].reset_index(drop=True),
    judge_parties.loc[j, :].reset_index(drop=True)
    ], axis=1).append(
    cases[~np.in1d(np.arange(len(cases)), np.unique(i))],
    ignore_index=True, sort=False
)


AttributeError: 'DataFrame' object has no attribute 'append'